In [1]:
import tensorflow as tf
from tensorflow import keras
import keras_cv
from tensorflow.keras import layers
from tensorflow.keras.utils import image_dataset_from_directory
import tensorflow_datasets as tfds
from keras_cv import utils
from keras_cv.layers import BaseImageAugmentationLayer
from tensorflow.keras import applications
from tensorflow.keras import losses
from tensorflow.keras import optimizers
import matplotlib.pyplot as plt
import numpy as np

In [2]:
physical_devices = tf.config.list_physical_devices('GPU')
print("Num GPUs:", len(physical_devices))

Num GPUs: 1


2022-11-04 16:22:57.887310: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-04 16:22:57.904838: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-04 16:22:57.904951: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [3]:
ds_train = image_dataset_from_directory(
'tiny_imagenet/train',
image_size=(64,64),
batch_size=32)

Found 100000 files belonging to 200 classes.


2022-11-04 16:23:00.673178: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-04 16:23:00.673331: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-04 16:23:00.673427: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-04 16:23:00.987625: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-04 16:23:00.987749: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from S

In [4]:
ds_val = image_dataset_from_directory(
'tiny_imagenet/val',
image_size=(64,64),
batch_size=32)

Found 10000 files belonging to 200 classes.


In [5]:
class RandomBlend(keras_cv.layers.BaseImageAugmentationLayer):
    """RandomBlend randomly applies a Multiply or Screen Blend to the images.

    Args:
      value_range: value_range: a tuple or a list of two elements. The first value
        represents the lower bound for values in passed images, the second represents
        the upper bound. Images passed to the layer should have values within
        `value_range`.
      factor: A tuple of two floats, a single float or a
        `keras_cv.FactorSampler`. `factor` controls the extent to which the
        image is Blend. `factor=0.0` makes this layer perform a no-op
        operation, while a value of 1.0 uses the degenerated result entirely.
        Values between 0 and 1 result in linear interpolation between the original
        image and the multiply or blended image.
        Values should be between `0.0` and `1.0`.  If a tuple is used, a `factor` is
        sampled between the two values for every image augmented.  If a single float
        is used, a value between `0.0` and the passed float is sampled.  In order to
        ensure the value is always the same, please pass a tuple with two identical
        floats: `(0.5, 0.5)`.
      blend_stack: An interger value that controls the amount of times the image is blended 
        to itself.  A value of 1 allows for the images to be blended once.  In most 
        cases a value of '1' to '3' will give the best results.
    """
    
    def __init__(self, value_range, factor, blend_stack, **kwargs):
        super().__init__(**kwargs)
        self.value_range = value_range
        self.factor = utils.parse_factor(factor)
        self.blend_stack = blend_stack
        self.auto_vectorize = False

    def get_random_transformation(self, **kwargs):
        # kwargs holds {"images": image, "labels": label, etc...}
        return self.factor() 

    def augment_image(self, image, transformation=None, **kwargs):

        # If blend_stack is set to '0' just return the image.
        if(self.blend_stack == 0):
            return image

        # Convert the image to values between '0' and '1'
        image = utils.transform_value_range(image, self.value_range, (0, 1))

        # Get a random value, either '0' or '1', to decide if screen of multiply will be performed. 
        # '0' for multiple. '1' for screen.
        multOrScreen = (np.random.randint(2))
        
        # If '0' perform multiply.
        if(multOrScreen==0):

            augImg = image * image
            # decrement blend_stack
            self.blend_stack = self.blend_stack - 1
            # While there are still more layers to blend.
            while(self.blend_stack > 0):
                augImg = augImg * image
                self.blend_stack = self.blend_stack - 1
            
        #  Else perform screen blend.
        else:

            augImg = 1 - (1 - image) * (1 - image)
            # decrement blend_stack
            self.blend_stack = self.blend_stack - 1
            # While there are still more layers to blend.
            while(self.blend_stack > 0):
                augImg = 1 - (1 - augImg) * (1 - image)
                self.blend_stack = self.blend_stack - 1

        # Take the augmented image and blend it back with the original.  Transform is the 
        # random value between the two factors supplied by the user.
        image = (augImg * transformation) + (image * (1-transformation))

        # Make sure there is not any image overflow.
        image = tf.clip_by_value(image, 0.0, 1.0)
        
        # Return the image. 
        return image

    def augment_label(self, label, transformation=None, **kwargs):
        return label

    def augment_bounding_boxes(self, bounding_boxes, transformation=None, **kwargs):
        return bounding_boxes

In [6]:
augmenter = keras_cv.layers.Augmenter(
  layers=[
      # keras.layers.Rescaling(scale=1./255),
      keras_cv.layers.RandomFlip(),
      # keras.layers.RandomTranslation(height_factor=0.2,width_factor=0.2),
      RandomBlend(value_range=(0, 1), factor=(0.01, 1.0), blend_stack=(2)),
      # keras_cv.layers.RandAugment(value_range=(0, 255)),
      # keras_cv.layers.CutMix(),
      # keras_cv.layers.MixUp()
    ]
)

normalization_layer = tf.keras.layers.Rescaling(1./255)

def augment_data(images, labels):
  inputs = {"images": images, "labels": labels}
  outputs = augmenter(inputs)
  return outputs['images'], outputs['labels']


def label_one_hot(images, labels):
  labels = tf.one_hot(labels, 200)
  outputs = {"images": images, "labels": labels}
  return outputs['images'], outputs['labels']

In [7]:
ds_train = ds_train.map(lambda x, y: (normalization_layer(x), y))

In [8]:
ds_val = ds_val.map(lambda x, y: (normalization_layer(x), y))

In [9]:
ds_train = ds_train.map(label_one_hot, num_parallel_calls=tf.data.AUTOTUNE)
ds_val = ds_val.map(label_one_hot, num_parallel_calls=tf.data.AUTOTUNE)

In [10]:
ds_train = ds_train.map(augment_data, num_parallel_calls=tf.data.AUTOTUNE)

In [11]:
# inputs = keras.Input(shape=(64,64,3))
# x = layers.Conv2D(filters=32, kernel_size=5, use_bias=False)(inputs)

# for size in [32,64,128,256,512]:
#     residual = x
    
#     x = layers.BatchNormalization()(x)
#     x = layers.Activation("relu")(x)
#     x = layers.SeparableConv2D(size, 3, padding="same", use_bias=False)(x)

#     x = layers.BatchNormalization()(x)
#     x = layers.Activation("relu")(x)
#     x = layers.SeparableConv2D(size, 3, padding="same", use_bias=False)(x)

#     x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

#     residual = layers.Conv2D(size, 1, strides=2, padding="same", use_bias=False)(residual)
#     x = layers.add([x, residual])


# x = layers.Flatten()(x)
# x = layers.Dropout(0.3)(x)
# x = layers.Dense(512, activation="relu")(x)
# x = layers.Dropout(0.3)(x)
# x = layers.Dense(256, activation="relu")(x)
# x = layers.Dropout(0.3)(x)
# x = layers.Dense(128, activation="relu")(x)
# x = layers.Dropout(0.3)(x)
# outputs = layers.Dense(200, activation="softmax")(x)
# model = keras.Model(inputs, outputs)
# model.summary()

In [12]:
# opt = tf.optimizers.Adam(learning_rate=0.0001)

# model.compile(
#   loss='categorical_crossentropy',
#   optimizer=opt,
#   metrics=['accuracy']
# )

In [13]:
# history = model.fit(ds_train, batch_size=32,validation_data=ds_val, epochs=30)

In [14]:
# history2 = model.fit(ds_train, batch_size=32,validation_data=ds_val, epochs=30)

In [15]:
# initial_learning_rate = 0.001
# lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
#     initial_learning_rate,
#     decay_steps=100000,
#     decay_rate=0.9,
#     staircase=True)

# epochs = 100
# learning_rate = 0.01
# decay_rate = learning_rate / epochs
# momentum = 0.9
# sgd = tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=momentum, decay=decay_rate, nesterov=False)

model = keras_cv.models.DenseNet121(
    include_rescaling=False, include_top=True, weights=None, input_shape=(64,64,3), classes=200)
    
model.compile(
   loss=losses.CategoricalCrossentropy(),
   optimizer=optimizers.SGD(momentum=0.9),
   metrics=["accuracy"],
)

In [16]:

history = model.fit(
    ds_train,
    epochs=100,
    batch_size=32,
    validation_data=ds_val,
)

Epoch 1/100


2022-11-04 16:23:08.983461: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8303
2022-11-04 16:23:10.632624: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


3125/3125 [==============================] - 109s 32ms/step - loss: 4.6282 - accuracy: 0.0654 - val_loss: 4.1429 - val_accuracy: 0.1116
Epoch 2/100
3125/3125 [==============================] - 108s 34ms/step - loss: 3.8315 - accuracy: 0.1599 - val_loss: 3.6844 - val_accuracy: 0.1812
Epoch 3/100
3125/3125 [==============================] - 106s 34ms/step - loss: 3.4390 - accuracy: 0.2217 - val_loss: 3.5653 - val_accuracy: 0.2078
Epoch 4/100
3125/3125 [==============================] - 107s 34ms/step - loss: 3.1709 - accuracy: 0.2702 - val_loss: 3.3689 - val_accuracy: 0.2460
Epoch 5/100
3125/3125 [==============================] - 106s 34ms/step - loss: 2.9663 - accuracy: 0.3073 - val_loss: 2.9782 - val_accuracy: 0.3031
Epoch 6/100
3125/3125 [==============================] - 109s 35ms/step - loss: 2.8025 - accuracy: 0.3363 - val_loss: 2.9784 - val_accuracy: 0.3094
Epoch 7/100
3125/3125 [==============================] - 107s 34ms/step - loss: 2.6517 - accuracy: 0.3649 - val_loss: 2.9147

In [17]:
np.save('tinyimagenet_densenet121_blend_flip_32batch_100epochs.npy',history.history)

In [18]:
# model.save('tiny_imagenet_flip_blend_densenet_epoch_100_150')

In [19]:
# Train/Test Accuracy Plot
# plt.plot(history.history['accuracy'])
# plt.plot(history.history['val_accuracy'])
# plt.title('Tiny Imagenet - Densenet121 - No Aug - Val: 38.45%')
# plt.ylabel('accuracy')
# plt.xlabel('epoch')
# plt.legend(['train', 'test'], loc='upper left')
# # plt.savefig('tiny_imagenet_no_aug.pdf')
# plt.show()